# constructing 2D/3D local box using GeoPoints.jl 

Nobuaki Fuji (IPGP/UPC/IUF) December 2025

In [ ]:
using Pkg

cd(@__DIR__)
Pkg.activate("../")
ParamFile = "../config/testparam.csv"


include("../src/planet1D.jl")
include("../src/GeoPoints.jl")

using .planet1D, .GeoPoints

In [ ]:
set_default_planet!(:Earth) # if you wish to go to other telluric bodies, otherwise this is not necessary

In [ ]:
# two (extreme) points that can define the slice (or the x-y local plane for 3D box)
p1 = GeoPoint(48.8566,2.3522) # Paris (latitude, longitude)
p2 = GeoPoint(42.8,1.5) # Tarascon (à peu près)


Δx = 100.0 # in metre
Δz = 100.0

altMax = 20.e3 # in metre
altMin = -100.e3 # in metre

In [ ]:
@show p1 # GeoPoint has some attributes

In [ ]:
# make a box/rectangle

allGridsInGeoPoints, allGridsInCartesian, effectiveRadii=constructLocalBox(p1,p2,Δx,Δz,altMin,altMax)


In [ ]:
allGridsInCartesian[20,30]

In [ ]:
seismicModel=getParamsAndTopo(allGridsInGeoPoints,effectiveRadii,2.0)

In [ ]:
Nx,Nz=size(allGridsInCartesian)

In [ ]:
maximum(seismicModel.ρ)

In [ ]:
using CairoMakie
fig, ax, hm = heatmap(
    #topo.x,topo.y,topo.z';
    collect((0:1:(Nx-1)).*Δx).*1.e-3,(collect(0:1:(Nz-1)).*Δz.+altMin).*1.e-3, seismicModel.ρ;
    colormap = :seismic,
    #colorrange=(0,4),
    axis = (aspect = DataAspect(), xlabel = "horizontal", ylabel = "depth from p1", title = "density model")
)
Colorbar(fig[1,2], hm, label="density")
fig

# 3D box ? yes

In [ ]:
p1 = GeoPoint(48.8566,2.3522) # Paris (latitude, longitude)
p2 = GeoPoint(42.8,1.5) # Tarascon (à peu près)


Δx = 500.0 # in metre
Δy = 500.0
Δz = 500.0

altMax = 20.e3 # in metre
altMin = -100.e3 # in metre

horizontalDepth = 50.e3

In [ ]:

allGridsInGeoPoints3D, allGridsInCartesian3D, effectiveRadii3D=constructLocalBox(p1,p2,Δx,Δy,Δz,-horizontalDepth,horizontalDepth,altMin,altMax)

#ok 3D box version needs to be GPUed

In [ ]:
seismicModel3D=getParamsAndTopo(allGridsInGeoPoints3D,effectiveRadii3D,2.0)

In [ ]:
using GLMakie
GLMakie.activate!()
Makie.inline!() 
Nx3D,Ny3D,Nz3D=size(allGridsInCartesian3D)

x = (0:Nx3D-1) .* Δx .* 1e-3
y = (0:Ny3D-1) .* Δy .* 1e-3 .- horizontalDepth*1.e-3
z = (0:Nz3D-1) .* Δz .* 1e-3 .+ altMin*1.e-3
A = seismicModel3D.Vpv

f = Figure()
ax = Axis3(f[1, 1])

volume!(ax,
    x[1] .. x[end],
    y[1] .. y[end],
    z[1] .. z[end],
    seismicModel3D.Vpv,
    algorithm = :absorption,   # optional, makes it nicer
    colormap = :viridis
)
f

# another way of importing model parameters

## we can read images or cartoons

In [ ]:
modelName="marmousi"
imageFile="../dataInput/model/random/marmousi.png"
modelDefinitionMethod="2DimageFile" # ToyModel or 2DimageFile (or 1DsphericalPlanet)
model=defineModel(imageFile)

here, i try to use some of heritages from demo1.ipynb (flex in myOwnApplications in the old version)

# now construction of OPT operators

# Let's go to Mars

In [ ]:
set_default_planet!(:Mars)


# ok i need to change how to call 1D planet models too (because it is already called by DSM1D and it is not very much flexible but here I just use the same params as Earth)

In [ ]:
p1 = GeoPoint(15.0,135.0) # we should see Elysium planitia at least
p2 = GeoPoint(15.0,180.0) # 

In [ ]:
Δx = 3.e3 # in metre
Δy = 3.e3
Δz = 3.e3

horizontalDepthMin = -1000.e3
horizontalDepthMax = 1000.e3


altMax = 100.e3 # in metre
altMin = -600.e3 # in metre



In [ ]:
allGridsInGeoPoints, allGridsInCartesian, effectiveRadii=constructLocalBox(p1,p2,Δx,Δy,Δz,horizontalDepthMin,horizontalDepthMax,altMin,altMax)

In [ ]:
Nx,Ny,Nz=size(allGridsInCartesian)

In [ ]:
seismicModel=getParamsAndTopo(allGridsInGeoPoints,2.0)

In [ ]:
using CairoMakie
fig, ax, hm = heatmap(
    #topo.x,topo.y,topo.z';
    collect((0:1:(Nx-1)).*Δx).*1.e-3,(collect(0:1:(Nz-1)).*Δz.+altMin).*1.e-3, seismicModel.Vsv[:,(Ny÷6)*1,:];
    colormap = :seismic,
    colorrange=(0,15),
    axis = (aspect = DataAspect(), xlabel = "horizontal", ylabel = "depth from p1", title = "Vph model")
)
ylims!(ax,-200,300)
xlims!(ax,0,400)
Colorbar(fig[1,2], hm, label="P-wave")
fig